# Design of Doors and Windows

### Window Model

![](images/window_model.png)

### Door Model

![](images/door_model.jpg)

### Code


In [2]:
from pyplasm import *
from larlib import *

"""
    Wrapper to COLOR with RGBA 0-255 scale
    
    Args:
        r:  the value of the red channel. An 
            integer between 0 and 255.
        b:  the value of the blue channel. An 
            integer between 0 and 255.
        g:  the value of the green channel. An 
            integer between 0 and 255.
        a:  the value of the alpha channel. A
            float between 0 and 1.
            Default value: 1
    
    Returns
        The COLOR function with the desired color
"""
rgba = lambda r, g, b, a=1: COLOR([r / 255.0, g / 255.0, b /255.0, a])


def addNeg(l, x):
    """
        Negates the positive number x and adds it to the list l.
        If the last element of l is negative, then adds -x to it,
        otherwise appends -x to l.
        
        Args:
            x:  the number to negate and append
            l:  the list to append
        
        Returns:
            N/A
    """
    if l != [] and l[-1] < 0:
        l[-1] -= x
    else:
        l.append(-x)


def glassStruct(X, Y, Z, occupancy, (glassColor, structColor)):
    """
        Creates a glass structure described by the input args.
        
        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
            glassColor: glass color
            structColor: struct color
        Returns
            An HPC value representing the desired structure
    """

    def aux(dx, dy, dz):
        """
            glassStruct auxiliary function. Creates the glass
            structure and scales it by the parameters
            Args:
                dx: scaling value (X-axis)
                dy: scaling value (Y-axis)
                dz: scaling value (Z-axis)
            Returns
                An HPC value representing the desired structure,
                scaled accordingly
        """
        prodX = lambda *args: reduce(lambda x, y: PROD([x, y]), args)
        struct = []
        glass = []
        for iY in range(0, len(Y)):
            structV = []
            glassV = []
            for iX in range(0, len(X)):
                value = X[iX]
                if occupancy[iY][iX] == 1:
                    structV.append(value)
                    addNeg(glassV, value)
                else:
                    addNeg(structV, value)
                    glassV.append(value)

            t = T([2])(Y[iY])
            if len(structV) > 1:
                struct.append(prodX(QUOTE(structV), QUOTE([Y[iY]]), QUOTE([Z[1]])))

            if len(glassV) > 1:
                glass.append(prodX(QUOTE(glassV), QUOTE([Y[iY]]), QUOTE([Z[0]])))

            struct.append(t)
            glass.append(t)

        final_struct = STRUCT(
            [
                T(3)(Z[1] / 2.0),
                rgba(*glassColor)(STRUCT(glass)),
                T(3)(-(Z[1] / 2.0)),
                rgba(*structColor)(STRUCT(struct))
            ])

        return STRUCT([S([1, 2, 3])([dx, dy, dz]), final_struct])
    return aux


def ggpl_window(X, Y, Z, occupancy):
    """
        Creates a window described by the input args.
        
        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
        Returns
            An HPC value representing a window
    """
    return glass_struct(X, Y, Z, occupancy, ((182, 208, 249), (255, 255, 255)))


def ggpl_door(X, Y, Z, occupancy):
    """
        Creates a door described by the input args.

        Args:
            X:         float list of lateral quotes (X-axis)
            Y:         float list of lateral quotes (Y-axis)
            Z:         float list of lateral quotes (Z-axis).
                       Only the first two values are used:
                       The first value for the glass depth,
                       the second value for the struct depth
            occupancy: a list of integer lists representing 
                       the incidence matrix of X on Y.
                       If occupancy[i][j] is 1, then the struct
                       is present, else there is glass
        Returns
            An HPC value representing a door
    """
    return glass_struct(X, Y, Z, occupancy, ((182, 208, 249), (0, 0, 0)))

#### Input
##### Window


In [3]:
window = [
    # X
    [0.07, 0.15, 0.015, 0.15,
     0.015, 0.15, 0.07],
    # Y
    [0.07, 0.18, 0.015, 0.18,
     0.07, 0.18, 0.015, 0.18,
     0.07],
    # Z
    [0.01, 0.05],
    # occupancy
    [
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
    ]
]

##### Door


In [5]:
door = [
    # X
    [0.0125, 0.5, 0.125, 0.025,
     0.125, 0.5, 0.0125],
    # Y
    [0.025, 0.3, 0.0125, 0.3,
     0.0125, 0.3, 0.0125, 1.2,
     0.025],
    # Z
    [0.01, 0.07],
    # occupancy
    [
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1]
    ]
]

#### Output

##### Window


In [10]:
VIEW(ggpl_window(*window)(1, 1, 1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2abcc4de40> >


![](images/window_out1.png)


In [11]:
VIEW(ggpl_window(*window)(1, 1.5, 1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2abcc4d780> >


![](images/window_out2.png)

##### Door


In [12]:
VIEW(ggpl_door(*door)(1, 1, 1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2abcc4dcc0> >


![](images/door_out1.png)


In [14]:
VIEW(ggpl_door(*door)(1, 1.3, 2.3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2abcc4d4b0> >


![](images/door_out2.png)
